# Test Metrics

In this Notebook the test metrcis are generated

### import

In [36]:
import pandas as pd
import numpy as np
import json
from scipy.spatial import distance

### load data

In [ ]:
# load yolo data
yolo_data = pd.read_csv('../results/Pre_Results_YOLO.csv')

In [ ]:
# read annotation json
with open('../../00_Dataset/annotations_ball.json') as f:
    data_test = json.load(f)

In [ ]:
def scale_points(point, scale):
    # scale and round to int
    return (int(point[0] * scale), int(point[1] * scale))

def convert_json_to_df(json_data, split='train', resolution=[1280, 720], file_path='../../00_Dataset/'):
    rows = []
    for subset in json_data['subsets']:
        resolution_subset= subset['resolution']
        if 'ball' in subset['objects']:
            for video in subset['videos']:
                for clip in video['clips']:
                    for frame_number, frame in clip['frames_with_objects'].items():    
                        if frame['split'] != split:
                            continue
                        points = []
                        for ball in frame['balls']:
                            if ball['visibility'] not in ['Outside'] and ball['trajectory'] not in ['', 'Static']:
                                point = scale_points([ball['x'],ball['y']], resolution[0] / resolution_subset[0])
                                point_object = {'x': point[0], 'y': point[1], 'visibility': ball['visibility']}
                                points.append(point_object)

                            
                        file_path_full = file_path + subset['name'] + '/' + video['name'] + '/' + clip['name'] + '/' + frame_number + '.jpg'
                        rows.append({
                            'subset': subset['name'],
                            'video': video['name'],
                            'clip': clip['name'],
                            'frame': int(frame_number), 
                            'file_path': file_path_full, 
                            'points': points
                        })

    return pd.DataFrame(rows)

# convert json to df
df_test = convert_json_to_df(data_test, split='test')

In [77]:
# split path into subset, video, clip, frame
yolo_data['subset'] = yolo_data['path'].apply(lambda x: x.split('_')[0])

# if subset = New we have more _ in the path
yolo_data['video'] = yolo_data.apply(lambda x: "_".join(x['path'].split('_')[1:3]) if x['subset'] == 'New' else x['path'].split('_')[1], axis=1)
yolo_data['clip'] = yolo_data.apply(lambda x: "_".join(x['path'].split('_')[3:5]) if x['subset'] == 'New' else x['path'].split('_')[2], axis=1)
yolo_data['frame'] = yolo_data.apply(lambda x: int(x['path'].split('_')[5].split('.')[0]) if x['subset'] == 'New' else int(x['path'].split('_')[3].split('.')[0]), axis=1)

yolo_data.sample(10)

,path,detections,model,subset,video,clip,frame
6154,New_Video_2_clip_7_172.jpg,[],tuned_yolov5su,New,Video_2,clip_7,172
14466,New_Video_1_clip_6_181.jpg,"[{'x1': 315.58465576171875, 'y1': 245.97631835...",tuned_yolo11s,New,Video_1,clip_6,181
27192,New_Video_5_clip_5_626.jpg,"[{'x1': 528.3784790039062, 'y1': 237.030914306...",yolo11s,New,Video_5,clip_5,626
24253,New_Video_4_clip_7_52.jpg,"[{'x1': 563.3439331054688, 'y1': 207.036346435...",yolov10l,New,Video_4,clip_7,52
20973,New_Video_3_clip_8_166.jpg,"[{'x1': 457.1470947265625, 'y1': 305.116973876...",yolov5su,New,Video_3,clip_8,166
19890,TrackNet_game8_Clip8_308.jpg,"[{'x1': 381.11163330078125, 'y1': 183.79901123...",yolov8s,TrackNet,game8,Clip8,308
16049,New_Video_6_clip_1_1.jpg,[],tuned_yolo11s,New,Video_6,clip_1,1
12435,New_Video_3_clip_8_25.jpg,"[{'x1': 456.5537109375, 'y1': 303.868347167968...",yolov10m,New,Video_3,clip_8,25
21633,New_Video_5_clip_6_243.jpg,"[{'x1': 596.2745971679688, 'y1': 304.077148437...",yolov5su,New,Video_5,clip_6,243
7776,TrackNet_game4_Clip7_211.jpg,"[{'x1': 752.1676025390625, 'y1': 101.794563293...",tuned_yolov5su,TrackNet,game4,Clip7,211


In [83]:
# join on subset, video, clip, frame
df_joined = pd.merge(df_test, yolo_data, on=['subset', 'video', 'clip', 'frame'])
df_joined.sample(10)

,subset,video,clip,frame,file_path,points,path,detections,model
7574,New,Video_3,clip_7,259,../FinalDataset/New/Video_3/clip_7/259.jpg,"[{'x': 755, 'y': 274, 'visibility': 'Difficult'}]",New_Video_3_clip_7_259.jpg,"[{'x1': 744.576416015625, 'y1': 265.2064819335...",yolov8s
20727,New,Video_6,clip_1,221,../FinalDataset/New/Video_6/clip_1/221.jpg,"[{'x': 388, 'y': 250, 'visibility': 'Difficult'}]",New_Video_6_clip_1_221.jpg,"[{'x1': 376.4606628417969, 'y1': 238.479400634...",yolov10s
1017,New,Video_1,clip_6,145,../FinalDataset/New/Video_1/clip_6/145.jpg,"[{'x': 398, 'y': 567, 'visibility': 'Visible'}]",New_Video_1_clip_6_145.jpg,"[{'x1': 387.9778747558594, 'y1': 556.753417968...",tuned_yolo11s
21797,TrackNet,game4,Clip7,170,../FinalDataset/TrackNet/game4/Clip7/170.jpg,"[{'x': 582, 'y': 200, 'visibility': 'Visible'}]",TrackNet_game4_Clip7_170.jpg,"[{'x1': 571.5987548828125, 'y1': 190.840087890...",yolov8s
4387,New,Video_2,clip_7,145,../FinalDataset/New/Video_2/clip_7/145.jpg,"[{'x': 522, 'y': 200, 'visibility': 'Visible'}]",New_Video_2_clip_7_145.jpg,"[{'x1': 564.28125, 'y1': 159.97732543945312, '...",yolo11s
28249,TrackNet,game6,Clip4,263,../FinalDataset/TrackNet/game6/Clip4/263.jpg,"[{'x': 624, 'y': 301, 'visibility': 'Difficult'}]",TrackNet_game6_Clip4_263.jpg,[],yolov9s
2538,New,Video_1,clip_6,283,../FinalDataset/New/Video_1/clip_6/283.jpg,[],New_Video_1_clip_6_283.jpg,[],yolov10l
16513,New,Video_5,clip_6,168,../FinalDataset/New/Video_5/clip_6/168.jpg,"[{'x': 608, 'y': 300, 'visibility': 'Difficult'}]",New_Video_5_clip_6_168.jpg,[],tuned_yolov5su
13617,New,Video_4,clip_7,222,../FinalDataset/New/Video_4/clip_7/222.jpg,"[{'x': 576, 'y': 285, 'visibility': 'Visible'}]",New_Video_4_clip_7_222.jpg,"[{'x1': 566.7197265625, 'y1': 275.092864990234...",yolov10x
15300,New,Video_5,clip_6,57,../FinalDataset/New/Video_5/clip_6/57.jpg,"[{'x': 654, 'y': 254, 'visibility': 'Visible'}]",New_Video_5_clip_6_57.jpg,"[{'x1': 637.8987426757812, 'y1': 234.115417480...",yolov10x


## Generate metrics

In [66]:
def preprocess_detections(detections):
    # Ensure the string uses double quotes for JSON compatibility
    if isinstance(detections, str):
        detections = detections.replace("'", '"')  # Replace single quotes with double quotes
    return detections

# calculae metrics for row 
def calculate_metrics(row, threshold=10):
    actual_position = (row['points'][0]['x'], row['points'][0]['y']) if len(row['points']) > 0 else (-1,-1)
    detections = row['detections']
    # convert detections string to list
    detections = json.loads(preprocess_detections(detections))
    if len(detections) == 0:
        predicted_position = (-1,-1)
    else:
        #get max confidence
        max_confidence = 0
        for detection in detections:
            if detection['confidence'] > max_confidence:
                max_confidence = detection['confidence']
                # calculate center of bounding box {'x1': 356.9684143066406, 'y1': 542.8724365234375, 'x2': 377.3059387207031, 'y2': 563.1761474609375, 'confidence': 0.7043583393096924, 'class_label': 0}
                predicted_position = ((detection['x1'] + detection['x2']) / 2, (detection['y1'] + detection['y2']) / 2)

    if actual_position == (-1,-1) and predicted_position == (-1,-1):
        # true negative
        return 0, 0, 1, 0
    elif actual_position == (-1,-1) and predicted_position != (-1,-1):
        # false positive
        return 0, 1, 0, 0
    elif actual_position != (-1,-1) and predicted_position == (-1,-1):
        # false negative
        return 0, 0, 0, 1
    else:
        if distance.euclidean(predicted_position, actual_position) < threshold:
            # true positive
            return 1, 0, 0, 0
        else:
            # false positive
            return 0, 1, 0, 0
        
# function to calculate complete metrics
def calculate_df_metrcis(frame_results_df, threshold=10):
    metrics_dict = {'Dataset_New': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Dataset_TrackNet': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_Visible': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_Difficult': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_Blurry': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_Occluded': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Visibility_None': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Video_6': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0},
                    'Total': {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0}}
    
    for i in range(len(frame_results_df)):
        row = frame_results_df.iloc[i]

        subset = "Dataset_" + row['subset']
        visibility = "Visibility_" + (row['points'][0]['visibility'] if  len(row['points']) > 0 else 'None')
        video = row['video']

        tp, fp, tn, fn = calculate_metrics(row, threshold)

        # subset metrics
        metrics_dict[subset]['tp'] += tp
        metrics_dict[subset]['fp'] += fp
        metrics_dict[subset]['tn'] += tn
        metrics_dict[subset]['fn'] += fn

        # visibility metrics
        metrics_dict[visibility]['tp'] += tp
        metrics_dict[visibility]['fp'] += fp
        metrics_dict[visibility]['tn'] += tn
        metrics_dict[visibility]['fn'] += fn

        #check if video is Video_1
        if video == 'Video_6':
            metrics_dict['Video_6']['tp'] += tp
            metrics_dict['Video_6']['fp'] += fp
            metrics_dict['Video_6']['tn'] += tn
            metrics_dict['Video_6']['fn'] += fn

        # total metrics
        metrics_dict['Total']['tp'] += tp
        metrics_dict['Total']['fp'] += fp
        metrics_dict['Total']['tn'] += tn
        metrics_dict['Total']['fn'] += fn
    
    return metrics_dict

def calculate_agg_metrics(tp, fp, tn, fn):
    # Calculate metrics
    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    metrics = {'Accuracy': accuracy, 'F1 Score': f1, 'Precision': precision, 'Recall': recall}
    return metrics

def calculate_positioning_error(frame_results_df, threshold=10):
    positioning_error = []
    for i in range(len(frame_results_df)):
        row = frame_results_df.iloc[i]
        actual_position = (row['points'][0]['x'], row['points'][0]['y']) if len(row['points']) > 0 else (-1,-1)
        detections = row['detections']
        # convert detections string to list
        detections = json.loads(preprocess_detections(detections))
        if detections == '[]':
            predicted_position = (-1,-1)
        else:
            #get max confidence
            max_confidence = 0
            for detection in detections:
                if detection['confidence'] > max_confidence:
                    max_confidence = detection['confidence']
                    # calculate center of bounding box {'x1': 356.9684143066406, 'y1': 542.8724365234375, 'x2': 377.3059387207031, 'y2': 563.1761474609375, 'confidence': 0.7043583393096924, 'class_label': 0}
                    predicted_position = ((detection['x1'] + detection['x2']) / 2, (detection['y1'] + detection['y2']) / 2)

        if actual_position != (-1,-1) and predicted_position != (-1,-1):
            error = distance.euclidean(predicted_position, actual_position)
            positioning_error.append(error)

    # get mean PE
    mean_positioning_error = np.mean(positioning_error)

    # get mean PE below TH
    mean_positioning_error_below_th = np.mean([pe for pe in positioning_error if pe < threshold])

    # count bins for 0, 1,2,3,4,1000
    bins = [0, 1, 2, 3, 4, 5, 10, 100000]

    # create counts
    counts = np.bincount(np.digitize(positioning_error, bins))
    
    return counts, mean_positioning_error, mean_positioning_error_below_th

In [84]:
# get distinct models
models = df_joined['model'].unique()
models

array(['yolov10n', 'yolov9s', 'tuned_yolov5su', 'yolov10s', 'yolov10m',
       'tuned_yolo11s', 'yolov8s', 'yolov5su', 'yolov10l', 'yolo11s',
       'yolov10x'], dtype=object)

In [ ]:
# calculate metrics
def get_stats_for_model(df):
    metrics_dict = calculate_df_metrcis(df)
    agg_metrics = {}
    for key, value in metrics_dict.items():
        tp = value['tp']
        fp = value['fp']
        tn = value['tn']
        fn = value['fn']

        agg_metrics[key] = calculate_agg_metrics(tp, fp, tn, fn)

    # filter for only f1-score
    agg_metrics_f1 = {key: value['F1 Score'] for key, value in agg_metrics.items()}

    # get total precision and recall
    precision = agg_metrics['Total']['Precision']
    recall = agg_metrics['Total']['Recall']
    
    # calculate positioning error
    counts, mean_positioning_error, mean_positioning_error_below_th = calculate_positioning_error(df)

    return agg_metrics_f1, precision, recall, mean_positioning_error, mean_positioning_error_below_th, counts

In [91]:
results = []
for model in models:
    df_model = df_joined[df_joined['model'] == model]
    agg_metrics_f1, precision, recall, mean_positioning_error, mean_positioning_error_below_th, counts = get_stats_for_model(df_model)
    
    # create object
    model_stats = {
        'Model': model,
        'precision': precision,
        'recall': recall,
        'mean_positioning_error': mean_positioning_error,
        'mean_positioning_error_below_th': mean_positioning_error_below_th,
        'counts': counts,
        'FPS': 0
    }
    # add stats from agg_metrics_f1
    for key, value in agg_metrics_f1.items():
        model_stats[key] = value

    results.append(model_stats)

results = pd.DataFrame(results)
results.head(1)

,Model,precision,recall,mean_positioning_error,mean_positioning_error_below_th,counts,FPS,Dataset_New,Dataset_TrackNet,Visibility_Visible,Visibility_Difficult,Visibility_Blurry,Visibility_Occluded,Visibility_None,Video_6,Total
0,yolov10n,0.816219,0.773885,29.168306,1.640241,"[0, 735, 654, 200, 79, 43, 89, 690]",0,0.755441,0.861871,0.887372,0.411972,0.552632,0.193548,0,0.791557,0.794489


In [94]:
# rename and filter columns
rename_dict = {
    'Model': 'Model',
    'Total': 'F1-Score Total',
    'precision': 'Precision Total',
    'recall': 'Recall Total',
    'Dataset_New': 'F1 Dataset New',
    'Dataset_TrackNet': 'F1 Dataset TrackNet',
    'Visibility_Visible': 'F1 Visibility Visibile',
    'Visibility_Difficult': 'F1 Visibility Difficult',
    'Visibility_Occluded': 'F1 Visibility Occluded',
    'Visibility_Blurry': 'F1 Visibility Blurry',
    'FPS': 'FPS',
    'mean_positioning_error': 'PE Mean',
    'mean_positioning_error_below_th': 'PE Mean Below Threshold',
    'counts': 'PE Counts',
}

results = results.rename(columns=rename_dict)

# filter columns
results = results[['Model', 'F1-Score Total', 'Precision Total', 'Recall Total', 'F1 Dataset New', 'F1 Dataset TrackNet', 'F1 Visibility Visibile', 'F1 Visibility Difficult', 'F1 Visibility Occluded', 'F1 Visibility Blurry', 'FPS', 'PE Mean', 'PE Mean Below Threshold', 'PE Counts']]
results.head(1)

,Model,F1-Score Total,Precision Total,Recall Total,F1 Dataset New,F1 Dataset TrackNet,F1 Visibility Visibile,F1 Visibility Difficult,F1 Visibility Occluded,F1 Visibility Blurry,FPS,PE Mean,PE Mean Below Threshold,PE Counts
0,yolov10n,0.794489,0.816219,0.773885,0.755441,0.861871,0.887372,0.411972,0.193548,0.552632,0,29.168306,1.640241,"[0, 735, 654, 200, 79, 43, 89, 690]"


In [ ]:
results.to_csv('../results/Results_YOLO.csv', index=False)